## 1) Localisation and hours

In [1]:
import os
from dotenv import load_dotenv

load_dotenv("C:/Users/busch/OneDrive/Documents/Fac/M2/UE1 - Advanced programming and data visualization/Advanced programming/projet/environment/.env")
hf_token = os.getenv("HUGGING_FACE_KEY")
custom_cache_dir = "/home/peltouz/Documents/pretrain"

os.environ['HF_HOME'] = custom_cache_dir  # Hugging Face home directory for all HF operations
os.environ['TRANSFORMERS_CACHE'] = custom_cache_dir  # Transformers-specific cache directory
os.environ['HF_DATASETS_CACHE'] = custom_cache_dir  # Datasets-specific cache directory
os.environ['HF_METRICS_CACHE'] = custom_cache_dir  # Metrics-specific cache directory
os.environ['HF_TOKEN'] = hf_token  # Hugging Face API token

In [11]:
import logging
import torch
from transformers import AutoModelForTokenClassification, pipeline, AutoTokenizer

In [7]:
logging.getLogger("transformers.modeling_utils").setLevel(logging.ERROR)
def HF_model(model, message):
    if model == "hours":
        m = AutoModelForTokenClassification.from_pretrained("DAMO-NLP-SG/roberta-time_identification")
        tokenizer = AutoTokenizer.from_pretrained("DAMO-NLP-SG/roberta-time_identification")
        nlp = pipeline("ner", model=m, tokenizer=tokenizer, aggregation_strategy="simple")
        ner_results = nlp(message)
    if model == "loc":
        m = AutoModelForTokenClassification.from_pretrained("Babelscape/wikineural-multilingual-ner")
        tokenizer = AutoTokenizer.from_pretrained("Babelscape/wikineural-multilingual-ner")
        nlp = pipeline("ner", model=m, tokenizer=tokenizer, aggregation_strategy="simple")
        ner_results = nlp(message)
    
    return ner_results

In [8]:
HF_model("hours", "I'll be there on 30 November 2016")

[{'entity_group': 'TIME',
  'score': 0.9936938,
  'word': ' 30 November 2016',
  'start': 17,
  'end': 33}]

In [9]:
HF_model("hours", "I'll be there today")

[{'entity_group': 'TIME',
  'score': 0.99966395,
  'word': ' today',
  'start': 14,
  'end': 19}]

In [12]:
HF_model("loc", "I want to eat at Esplanade, what is the menu for the lunch")

[{'entity_group': 'LOC',
  'score': 0.99668074,
  'word': 'Esplanade',
  'start': 17,
  'end': 26}]

In [13]:
ent = HF_model("loc", "I want to eat at Esplanade, what is the menu for the lunch ?")
entity_group = ent[0]['entity_group']
resto = ent[0]['word']

In [14]:
resto

'Esplanade'

## 2) Text similarity

In [17]:
from sentence_transformers import SentenceTransformer
import torch

In [18]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [19]:
# Define two example sentences
sentence1 = "Esplonde"
sentence2 = "Esplanade"

In [20]:
# Encode sentences and compute similarity score
embeddings1 = model.encode([sentence1], convert_to_tensor=True)
embeddings2 = model.encode([sentence2], convert_to_tensor=True)
cosine_similarities = torch.nn.functional.cosine_similarity(embeddings1, embeddings2)

In [21]:
# Print similarity score
print(f"Similarity score: {cosine_similarities.item()}")

Similarity score: 0.8379175662994385


In [22]:
def resto_link(resto):
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    vec_resto = ["cafeteria le pege", "resto u gallia", "resto u esplanade", "resto u paul appell", "le 32", "lannexe", 
                 "resto u illkirch", "cafeteria mini r", "resto u cronenbourg", "le cristal shop ru esplanade"]
    link = []
    for i in range(len(resto)):
        sim_score = []
        for j in range(len(vec_resto)):
            embeddings1 = model.encode([resto[i]], convert_to_tensor=True)
            embeddings2 = model.encode([vec_resto[j]], convert_to_tensor=True)
            sim_score.append(torch.nn.functional.cosine_similarity(embeddings1, embeddings2).item())
        k = sim_score.index(max(sim_score))
        link.append("https://www.crous-strasbourg.fr/restaurant/" + vec_resto[k].replace(" ", "-") + "-2/")
    
    return link

## 3) Translator

In [3]:
def translator(language_from, language_to, text):
    if language_from == "fr" and language_to == "en":
        api_translator = pipeline("translation", model="Helsinki-NLP/opus-mt-fr-en")
        translation = api_translator(text)[0]['translation_text']
    return translation

In [1]:
from menu_functions import get_menu
menu = get_menu("Esplanade")

In [2]:
print(menu)

- Student menus 2nd floor

Chicken hash with homemade cream
Flat beans
Torsades

Chile with homemade vegetables
Vegetable soup of the day:
Beans, cauliflower and carrots

- Desserts 2nd floor

Paris-Brest
Chocolate tart
Coconut tart
Fruit salad

- Crystal Shop (DRC) Cafeteria

menu not communicated

- Origins of our meats of the day

Poultry origin France




## 4) Text analysis

In [15]:
menu

'- Student menus 2nd floor\n\nChicken hash with homemade cream\nFlat beans\nTorsades\n\nChile with homemade vegetables\nVegetable soup of the day:\nBeans, cauliflower and carrots\n\n- Desserts 2nd floor\n\nParis-Brest\nChocolate tart\nCoconut tart\nFruit salad\n\n- Crystal Shop (DRC) Cafeteria\n\nmenu not communicated\n\n- Origins of our meats of the day\n\nPoultry origin France\n\n'

In [18]:
question_answerer = pipeline("question-answering")
question_answerer(
    question="Where can i eat Chile ?",
    context=menu_str,
)

No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 564e9b5 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


{'score': 0.5917239785194397, 'start': 14, 'end': 23, 'answer': '2nd floor'}

In [10]:
question_answerer(
    question="What is my birthday ?",
    context="My name is Adrien and I was born the 3rd december",
)

{'score': 0.15749846398830414,
 'start': 37,
 'end': 49,
 'answer': '3rd december'}